---
title: "Security Incidents in Palestine: Trends and Patterns (2002-2025)"
format:
  html:
    toc: true
    toc-depth: 3
    code-fold: true
    number-sections: true
    theme: cosmo
    fig-width: 9
    fig-height: 6
    fig-dpi: 300
    self-contained: true
execute:
  warning: false
  message: false
---

In [ ]:
#| label: setup
#| include: false

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from matplotlib.colors import LinearSegmentedColormap
import warnings
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display
from scipy import stats

# Configure plot settings
plt.rcParams['figure.figsize'] = (10, 6)
warnings.filterwarnings('ignore', category=FutureWarning)
sns.set_theme(style="whitegrid")

# Set consistent color palette
palette = {"primary": "#FFBD59", "secondary": "#64C1FF", "danger": "#FF5252"}

# Load and prepare data
df = pd.read_csv("data/security_incidents_cleaned.csv")
palestine_data = df[df['country'] == 'Occupied Palestinian Territories']

# Check if data is available
if len(palestine_data) == 0:
    print("No data found for 'Occupied Palestinian Territories'")

![](images/palestine.png)

# Executive Summary

This analysis examines security incidents in the Occupied Palestinian Territories from 2002 to 2025, revealing significant patterns and trends. Key findings include:

-   **Dramatic escalation in 2023-2024**: Following the October 7, 2023 Hamas attack and subsequent Israeli military campaign, security incidents surged to unprecedented levels in 2024.
-   **Aerial bombardment** is the predominant attack method, reflecting the nature of military operations in the region.
-   **Palestinians bear the brunt** of the conflict, with nationals suffering significantly higher casualties than international personnel.
-   **Combat/crossfire** contexts account for the majority of incidents, highlighting the risks of active conflict zones.

These findings have critical implications for humanitarian operations, civilian protection strategies, and policy development in the region.

# Background: The Palestinian Conflict

```{}
```

## Historical Context

Palestine has experienced a complex and turbulent history of conflict that has significantly intensified in recent times. The current crisis stems from decades of territorial disputes and failed peace efforts:

-   **1948**: Israel's creation displaced hundreds of thousands of Palestinians
-   **1967**: Six-Day War led to Israeli control of Gaza Strip, West Bank, and East Jerusalem
-   **1990s**: Oslo Accords established limited Palestinian self-governance
-   **2006**: Hamas won Palestinian elections and took control of Gaza
-   **2014 & 2021**: Major confrontations between Hamas and Israel

## Recent Escalation (2023-2025)

The situation dramatically escalated on October 7, 2023, when Hamas launched a surprise attack on Israel, killing over 1,300 Israelis and taking hundreds hostage. Israel responded with a full-scale war in Gaza that has had devastating humanitarian consequences:

-   Approximately 90% of Gaza's population displaced
-   Over 46,000 Palestinian casualties reported
-   Destruction of essential infrastructure including hospitals
-   Severe humanitarian crisis with widespread food insecurity
-   Limited access to humanitarian aid

A ceasefire was brokered in January 2025, but its implementation remains uncertain amid ongoing political tensions.

# Temporal Trends in Security Incidents {#sec-temporal}

## Incident Frequency Over Time


In [ ]:
#| label: fig-yearly-incidents
#| fig-cap: Security incidents in Palestine from 2002 to 2025
#| echo: false

# Group incidents by year
yearly_incidents = palestine_data.groupby('year').size().reset_index(name='incidents')

# Create line chart
fig = px.line(
    yearly_incidents, 
    x='year', 
    y='incidents',
    markers=True,
    labels={'incidents': 'Number of Incidents', 'year': 'Year'},
    height=500
)

# Configure layout
fig.update_layout(
    title=None,
    xaxis=dict(tickmode='linear', dtick=2),
    yaxis=dict(zeroline=True),
    hovermode='x unified',
    template='plotly_white'
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

# Add event annotations
events = [
    {'year': 2006, 'event': 'Hamas takes control of Gaza', 'y_position': 5},
    {'year': 2014, 'event': 'Israel-Hamas conflict', 'y_position': 10},
    {'year': 2018, 'event': 'Gaza border protests', 'y_position': 8},
    {'year': 2021, 'event': '11-day conflict', 'y_position': 7},
    {'year': 2023, 'event': 'Oct 7 Hamas attack', 'y_position': 15},
    {'year': 2024, 'event': 'Gaza war escalation', 'y_position': 65}
]

for event in events:
    fig.add_annotation(
        x=event['year'],
        y=event['y_position'],
        text=event['event'],
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-20,
        font=dict(size=10),
        bgcolor="rgba(255,255,255,0.85)",
        bordercolor="black",
        borderwidth=1,
        borderpad=3
    )

fig.show()

### Analysis of Incident Trends

The line chart reveals a striking pattern in security incidents in the Occupied Palestinian Territories:

-   **Relative stability (2002-2022)**: For nearly two decades, incident levels remained relatively stable, typically below 10 incidents per year with occasional modest spikes around 2010, 2014, and 2020, corresponding to periods of heightened tension.

-   **Unprecedented surge (2023-2024)**: A dramatic shift occurred after the October 7, 2023 Hamas attack, with incidents surging to 68 in 2024 - representing approximately 43% of all recorded incidents in the entire dataset.

-   **Historical context**: While periodic cycles of violence have occurred historically, the 2023-2024 escalation represents a significant departure from previous patterns in both scale and intensity.

This finding highlights the unprecedented nature of the current conflict, which has fundamentally altered security dynamics throughout the Palestinian territories.

## Effected Indivudals Over Time


In [ ]:
#| label: fig-yearly-affected
#| fig-cap: Total affected individuals in Palestine from 2002 to 2025
#| echo: false

# Group by year and sum total_affected
yearly_affected = (
    palestine_data.groupby('year')['total_affected']
    .sum()
    .reset_index()
    .rename(columns={'total_affected': 'affected'})
)

# Create line chart
fig = px.line(
    yearly_affected, 
    x='year', 
    y='affected',
    markers=True,
    labels={'affected': 'Total Affected Individuals', 'year': 'Year'},
    height=500
)

# Configure layout
fig.update_layout(
    title=None,
    xaxis=dict(tickmode='linear', dtick=2),
    yaxis=dict(zeroline=True),
    hovermode='x unified',
    template='plotly_white'
)

# Add range slider
fig.update_xaxes(rangeslider_visible=True)

fig.show()

### Analysis of Humans Impacted

The chart above shows the total number of individuals affected by security incidents each year. The pattern closely mirrors the incident frequency trend, with a dramatic spike in 2023-2024 that dwarfs all previous years combined. This underscores the unprecedented humanitarian impact of the recent conflict.

# Nature of Security Incidents {#sec-nature}

## Predominant Attack Types


In [ ]:
#| label: fig-attack-types
#| fig-cap: Most common types of attacks in Palestine
#| echo: false

# Analyzing means of attack
attack_counts = palestine_data['means_of_attack'].value_counts().reset_index()
attack_counts.columns = ['Attack Type', 'Count']
attack_counts = attack_counts.sort_values('Count', ascending=False).head(10)

# Create horizontal bar chart with uniform color
fig_attacks = px.bar(
    attack_counts,
    y='Attack Type',
    x='Count',
    title=None,
    height=500,
    orientation='h'
)

# Set all bars to primary color
fig_attacks.update_traces(marker_color=palette["primary"])

# Configure layout
fig_attacks.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title="Number of Incidents",
    yaxis_title=None
)

fig_attacks.show()

### Analysis of Attack Methods

The horizontal bar chart reveals that **aerial bombardment** is by far the most common type of attack in the Occupied Palestinian Territories, followed by shooting incidents. This distribution offers important insights:

1.  **Military nature of conflict**: The predominance of aerial bombardment suggests a significant role of military forces with air capabilities in the conflict.

2.  **Diversity of threats**: Beyond the top two methods, the data shows a range of attack types including shelling, bodily assault, and various forms of explosives.

3.  **Civilian vulnerability**: Many of these attack types affect wide areas, putting civilians at significant risk even when not directly targeted.

This information is crucial for security planning, as different types of attacks require different prevention strategies and response protocols.

## Attack Contexts


In [ ]:
#| label: fig-attack-contexts
#| fig-cap: Security incidents by attack context
#| echo: false

# Get counts of incidents by attack context
context_counts = palestine_data['attack_context'].value_counts().reset_index()
context_counts.columns = ['Attack Context', 'Count']

# Sort by count for better visualization
context_counts = context_counts.sort_values('Count', ascending=False)

# Create a column chart
fig_context = px.bar(
    context_counts,
    x='Attack Context',
    y='Count',
    title=None,
    height=450
)

# Set primary color for all bars
fig_context.update_traces(marker_color=palette["primary"])

# Improve layout
fig_context.update_layout(
    xaxis_title=None,
    yaxis_title="Number of Incidents",
    xaxis={'categoryorder': 'total descending', 'tickangle': 45}
)

fig_context.show()

### Analysis of Attack Contexts

"Combat/Crossfire" is overwhelmingly the most common context for security incidents, accounting for over 100 incidents - far more than all other contexts combined. This finding has important implications:

1.  **Collateral impact**: Many incidents occur during active combat situations where individuals are caught between opposing forces rather than being directly targeted.

2.  **Conflict zone risks**: Being in or near active conflict zones represents the greatest security risk in the region.

3.  **Security planning**: Different contexts require different security approaches - avoiding combat zones entirely may be more effective than other security measures.

The significant number of "Individual attack" incidents (second most common) also suggests that targeted violence remains a notable security concern, requiring different mitigation strategies.

# Human Impact of Security Incidents {#sec-impact}

## Casualties by Category


In [ ]:
#| label: fig-casualties
#| fig-cap: Total casualties in Palestine by category
#| echo: false

# Create data for stacked bar chart
stacked_data = {
    'Category': ['Nationals', 'Internationals'],
    'Killed': [
        palestine_data['nationals_killed'].sum(),
        palestine_data['internationals_killed'].sum()
    ],
    'Wounded': [
        palestine_data['nationals_wounded'].sum(),
        palestine_data['internationals_wounded'].sum()
    ],
    'Kidnapped': [
        palestine_data['nationals_kidnapped'].sum(),
        palestine_data['internationals_kidnapped'].sum()
    ]
}

# Convert to DataFrame and reshape for stacking
stacked_df = pd.DataFrame(stacked_data)
stacked_df_long = pd.melt(
    stacked_df,
    id_vars=['Category'],
    value_vars=['Killed', 'Wounded', 'Kidnapped'],
    var_name='Status',
    value_name='Count'
)

# Create a stacked bar chart
fig_stacked = px.bar(
    stacked_df_long,
    x='Category',
    y='Count',
    color='Status',
    title=None,
    barmode='stack',
    height=500,
    color_discrete_map={
        'Killed': palette["danger"], 
        'Wounded': palette["primary"], 
        'Kidnapped': palette["secondary"]
    }
)

# Improve layout
fig_stacked.update_layout(
    xaxis_title=None,
    yaxis_title="Number of People",
    legend_title_text=None
)

fig_stacked.show()

### Analysis of Casualty Patterns

The stacked bar chart reveals a stark disparity in the impact of security incidents on different population groups:

-   **Disproportionate local impact**: Nationals (local Palestinians) have suffered significantly higher casualties compared to international personnel, in both fatalities and injuries.

-   **Minimal kidnapping incidents**: Kidnapping appears to be exceedingly rare, with only one recorded case involving international personnel and none for nationals.

-   **Scale of humanitarian crisis**: The high casualty figures for nationals underscore the severe humanitarian impact of the conflict on local populations.

This pattern is not unusual in conflict zones, where local populations typically bear the brunt of violence. However, the scale of the difference is notable and has important implications for civilian protection and humanitarian programming.

## Gender Distribution of Affected Individuals


In [ ]:
#| label: fig-gender
#| fig-cap: Gender distribution of affected individuals
#| echo: false

# Check if we have gender data
gender_cols = ['gender_male', 'gender_female']
if all(col in palestine_data.columns for col in gender_cols):
    # Sum up total counts by gender
    gender_totals = {
        'Gender': ['Male', 'Female'],
        'Count': [
            palestine_data['gender_male'].sum(),
            palestine_data['gender_female'].sum()
        ]
    }

    # Convert to DataFrame
    gender_df = pd.DataFrame(gender_totals)

    # Create simple bar chart
    fig_gender = px.bar(
        gender_df,
        x='Gender',
        y='Count',
        title=None,
        height=450
    )

    # Set primary color for bars
    fig_gender.update_traces(marker_color=palette["primary"])

    # Improve layout
    fig_gender.update_layout(
        xaxis_title=None,
        yaxis_title="Number of Individuals"
    )

    fig_gender.show()

### Analysis of Gender Patterns

The chart shows the gender distribution of individuals affected by security incidents, focusing only on confirmed male and female casualties. Males appear to be more frequently affected than females, which aligns with patterns observed in many conflict zones. This gender disparity can be influenced by several factors:

1.  **Gender roles and mobility**: Males may have higher exposure to security risks due to mobility patterns or engagement in certain activities.

2.  **Targeting patterns**: Some types of security incidents may affect certain gender groups disproportionately.

3.  **Data limitations**: It's important to note that data collection in conflict zones can sometimes undercount impacts on specific groups.

Understanding these gender dimensions is crucial for developing gender-sensitive security protocols and humanitarian responses.

# Organizations and Actors {#sec-organizations}

## Organizations Affected by Security Incidents


In [ ]:
#| label: fig-organizations
#| fig-cap: Security incidents by organization type
#| echo: false

# Count incidents affecting different organization types
org_cols = ['un', 'ingo', 'icrc', 'nrcs_and_ifrc', 'nngo', 'other']

# Create a summary of total incidents by organization type
org_totals = palestine_data[org_cols].sum().reset_index()
org_totals.columns = ['Organization Type', 'Total Incidents']

# Better labels for organization types
org_labels = {
    'un': 'United Nations',
    'ingo': 'International NGO',
    'icrc': 'Int. Committee of Red Cross',
    'nrcs_and_ifrc': 'National Red Cross/Red Crescent',
    'nngo': 'National NGO',
    'other': 'Other Organizations'
}

# Map the short names to full names
org_totals['Organization'] = org_totals['Organization Type'].map(org_labels)

# Sort by total incidents
org_totals = org_totals.sort_values('Total Incidents', ascending=False)

# Create a horizontal bar chart
fig_orgs = px.bar(
    org_totals,
    y='Organization',
    x='Total Incidents',
    title=None,
    height=500,
    orientation='h'
)

# Set all bars to primary color
fig_orgs.update_traces(marker_color=palette["primary"])

# Improve layout
fig_orgs.update_layout(
    yaxis_title=None,
    xaxis_title="Number of Incidents",
    yaxis={'categoryorder': 'total ascending'}
)

fig_orgs.show()

### Analysis of Organizational Impact

The chart shows how security incidents have affected different types of organizations operating in the Occupied Palestinian Territories. This distribution reveals several important patterns:

1.  **United Nations vulnerability**: UN organizations have been affected by the highest number of incidents, potentially reflecting their widespread presence and high visibility in the region.

2.  **International vs. local exposure**: International organizations appear more frequently affected than local entities, which may reflect differences in operational footprint, reporting practices, or targeting patterns.

3.  **Humanitarian access challenges**: The significant impact on humanitarian organizations has implications for service delivery to vulnerable populations.

These findings can help organizations develop security strategies tailored to their specific risk profiles and improve coordination between different actors operating in the region.

## Comparison of Actor Types


In [ ]:
#| label: fig-actors
#| fig-cap: Host State vs Foreign Actors in Security Incidents
#| echo: false

# Filter only for Host State and Foreign actors
relevant_actors = ['Host state', 'Foreign or coalition forces']
actor_data = palestine_data[palestine_data['actor_type'].isin(relevant_actors)]

# Check if we have data for these specific actors
if len(actor_data) > 0:
    actor_counts = actor_data['actor_type'].value_counts().reset_index()
    actor_counts.columns = ['Actor Type', 'Count']
    
    # Create a column chart
    fig_actors = px.bar(
        actor_counts,
        x='Actor Type',
        y='Count',
        title=None,
        height=450
    )
    
    # Set primary color for all bars
    fig_actors.update_traces(marker_color=palette["primary"])
    
    # Improve layout
    fig_actors.update_layout(
        xaxis_title=None,
        yaxis_title="Number of Incidents"
    )
    
    fig_actors.show()

### Analysis of Actor Responsibility

This analysis focuses specifically on two major actor types responsible for security incidents: Host State forces and Foreign/coalition forces. Other actor categories were excluded to enable a clear comparison between these two primary groups.

The chart clearly shows that host state forces are behind most of the security incidents.

# Major Incidents {#sec-major}

## Top 5 Most Impactful Security Incidents


In [ ]:
#| label: tbl-top-incidents
#| tbl-cap: Most impactful security incidents in Palestine (by total affected)
#| echo: false

if not palestine_data.empty and 'total_affected' in palestine_data.columns:
    top5_incidents = palestine_data.sort_values(by='total_affected', ascending=False).head(5)
    
    # Select and rename columns for display
    display_cols = ['year', 'location', 'total_affected', 'total_killed', 'total_wounded', 'attack_context', 'actor_type']
    column_names = {
        'year': 'Year', 
        'location': 'Location', 
        'total_affected': 'Total Affected',
        'total_killed': 'Killed', 
        'total_wounded': 'Wounded',
        'attack_context': 'Context',
        'actor_type': 'Actor Type'
    }
    
    display_df = top5_incidents[display_cols].rename(columns=column_names)
    display(display_df)

### Analysis of Major Incidents

The table above highlights the five most impactful security incidents in the Occupied Palestinian Territories based on the total number of people affected. The most severe incident occurred in October 2023, coinciding with the Hamas attack on October 7th and the subsequent Israeli military response.

These major incidents share several characteristics:

1.  **Recent escalation**: Most of the high-impact incidents occurred during the 2023-2024 conflict escalation.

2.  **High casualty rates**: These incidents resulted in significant casualties, underscoring the intensity of the recent conflict.

3.  **Combat contexts**: Most occurred in combat/crossfire situations, consistent with the broader pattern of incidents in the region.

For more detailed information on the Israeli-Palestinian conflict timeline, see the [Council on Foreign Relations Conflict Tracker](https://www.cfr.org/global-conflict-tracker/conflict/israeli-palestinian-conflict).

# Conclusions and Recommendations {#sec-conclusions}

## Key Findings

This analysis of security incidents in the Occupied Palestinian Territories reveals several critical patterns and trends:

1.  **Unprecedented recent escalation**: After nearly two decades of relatively stable incident levels, 2023-2024 saw a dramatic surge in security incidents, reflecting the intensity of the conflict following the October 7th attack.

2.  **Predominant attack methods**: Aerial bombardment and shooting incidents emerge as the most common types of attacks, highlighting the nature of military operations in the region.

3.  **Disproportionate impact on locals**: Palestinian nationals have suffered significantly higher casualties compared to international personnel, underscoring the humanitarian dimension of the conflict.

4.  **Combat as primary context**: Most incidents occur during active combat situations, suggesting that conflict avoidance is a critical security strategy.

5.  **Organizational vulnerability**: Various organizations operating in the region face different levels of security risk, with UN entities being most frequently affected.

## Recommendations

These findings have important implications for security planning, humanitarian programming, and policy development:

### For Security Planning

-   Develop robust protocols specifically addressing risks from aerial bombardment and crossfire situations
-   Implement enhanced early warning systems to monitor combat zone developments
-   Establish clear evacuation procedures for areas at high risk of becoming combat zones

### For Humanitarian Organizations

-   Prioritize protection measures for local staff, who face disproportionate security risks
-   Develop gender-sensitive security protocols that address the different risk profiles of male and female personnel
-   Strengthen coordination among different organizations to share security information and resources

### For Policy Development

-   Advocate for strengthened civilian protection measures, particularly in areas prone to combat/crossfire
-   Support accountability mechanisms for documenting and addressing security incidents
-   Promote conflict resolution initiatives to address the root causes of the escalating violence

### For Future Research

-   Continue monitoring security trends, especially given the recent escalation
-   Investigate the specific factors contributing to the disproportionate impact on local populations
-   Examine the effectiveness of different security measures in reducing casualties

By providing a data-driven understanding of security patterns in the Occupied Palestinian Territories, this analysis aims to contribute to safer and more effective operations in this challenging environment.